# Test 16

_Quench at 90 degrees_

The goal of this file is to get an idea on which field i need to use to order the system.

**Simulation with LAMMPS**

Simulations are done by a Ramp up to 20mT in 120s

In [ ]:
import os
import sys

import numpy as np
import pandas as pd

from tqdm import tqdm
from IPython.display import clear_output

import matplotlib as mpl 
#mpl.use('pgf')
import matplotlib.pyplot as plt

sys.path.insert(0, '../icenumerics/')
import icenumerics as ice

import auxiliary as aux
import montecarlo_tools as mc
import chirality_tools as chir
ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

In [ ]:
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# MD Simulations

This part only generates the datasets to work with for all angles.

In [ ]:
quench_time = 120*ureg.s
rotation_time = 0*ureg.s
total_time = quench_time + rotation_time
data_path = "../data/test16/"
runs = 10

params = {
    "particle_radius":1.4*ureg.um,
    "particle_susceptibility":0.4,
    "particle_diffusion":0.14*ureg.um**2/ureg.s,
    "particle_temperature":1000*ureg.K,
    "particle_density":1000*ureg.kg/ureg.m**3,

    "trap_sep":3*ureg.um,
    "trap_height":8*ureg.pN*ureg.nm,
    "trap_stiffness":100e-3*ureg.pN/ureg.nm,
    "height_spread":0,
    "susceptibility_spread":0,
    "isperiodic":True,

    "total_time":total_time,
    "framespersec":20*ureg.Hz,
    "dt":0.1*ureg.ms,
    "max_field":20*ureg.mT,
    "sim_temp":1000*ureg.K,
    "sim_dipole_cutoff":40*ureg.um,
}

params["lattice_constant"] = params["trap_sep"]+(2*params["particle_radius"]+1*ureg.um)*np.sqrt(2)
params["size"] = 10

#locals().update(params)

El cutoff debe de ser de 65 micras

In [ ]:
params['lattice_constant'] * params['size']

In [ ]:
def get_md_colloid(params,realization):
    sp = ice.spins()
    
    N = params["size"]
    a = params["lattice_constant"]
    
    # Create simulation objects
    sp.create_lattice("square",[N,N],lattice_constant=a, border="periodic")

    particle = ice.particle(radius = params["particle_radius"],
                susceptibility = params["particle_susceptibility"],
                diffusion = params["particle_diffusion"],
                temperature = params["particle_temperature"],
                density = params["particle_density"])

    trap = ice.trap(trap_sep = params["trap_sep"],
                height = params["trap_height"],
                stiffness = params["trap_stiffness"])

    col = ice.colloidal_ice(sp, particle, trap,
                            height_spread = params["height_spread"], 
                            susceptibility_spread = params["susceptibility_spread"],
                            periodic = params["isperiodic"])

        
    col.randomize()
    col.region = np.array([[0,0,-3*(params["particle_radius"]/a/N).magnitude],[1,1,3*(params["particle_radius"]/a/N).magnitude]])*N*a


    world = ice.world(
            field = params["max_field"],
            temperature = params["sim_temp"],
            dipole_cutoff = params["sim_dipole_cutoff"],
            boundaries=['p', 'p', 'p'])


    col.simulation(world,
                name = f"./lammps_files/test{realization}",
                include_timestamp = False,
                targetdir = r".",
                framerate = params["framespersec"],
                timestep = params["dt"],
                run_time = params["total_time"],
                output = ["x","y","z","mux","muy","muz"],
                processors=1)

    # Field
    

    col.sim.field.fieldx = "v_Bmag/120e6*time"
    col.sim.field.fieldy = "0"
    col.sim.field.fieldz = "0"        
        
    col.run_simulation()

    col.load_simulation()

    return col

In [ ]:
def jesus_take_the_wheels(params,data_path,realization):
    trj_path = data_path + "/trj/"
    ctrj_path = data_path + "/ctrj/"

    try:
        os.mkdir(trj_path)
        os.mkdir(ctrj_path)
    except:
        pass
    
    col = get_md_colloid(params,realization)

    filename = f"trj{realization}.csv"
    col.trj.to_csv(trj_path + filename)

    filename = f"ctrj{realization}.csv"
    trj = ice.get_ice_trj(col.trj, bounds = col.bnd)
    trj.to_csv(ctrj_path + filename)
    
    return None

# MD Sims

This block checks which angles are going to be ran. Helps for running by batches.

In [ ]:
import concurrent.futures

In [ ]:
runs = 5
realizations = list(range(1,runs+1))
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # map the function to the realizations
    results = list(
        executor.map(
            jesus_take_the_wheels,
            [params] * len(realizations),
            [data_path] * len(realizations),
            realizations,
        )
    )

This section graphs the fields just to check that everything is OK.

In [ ]:
trj_raw = trj = pd.read_csv("../data/test16/trj/trj1.csv", index_col=[0,1])
ctrj_raw = pd.read_csv("../data/test16/ctrj/ctrj1.csv", index_col=[0,1])

fig, axes = plt.subplots(1,3,figsize=(9,2),dpi=150)

ax = axes[0]
ax.plot(np.array(trj_raw.loc[idx[:,1],"t"]),np.array(trj_raw.loc[idx[:,1],"mux"]))
ax.set_title("$\\sim B_x$")
ax.ticklabel_format(style='sci',scilimits=(0,0))

ax = axes[1]
ax.plot(np.array(trj_raw.loc[idx[:,1],"t"]),np.array(trj_raw.loc[idx[:,1],"muy"]))
ax.set_title("$\\sim B_y$")
ax.ticklabel_format(style='sci',scilimits=(0,0))

ax = axes[2]
ax.plot(np.array(trj_raw.loc[idx[:,1],"t"]),np.array(trj_raw.loc[idx[:,1],"muz"]))
ax.set_title("$\\sim B_z$")
ax.ticklabel_format(style='sci',scilimits=(0,0))

plt.show()

# Vertex counting

In [ ]:
trj_path = os.path.join(data_path,"trj")
ctrj_path = os.path.join(data_path,"ctrj")
vrt_path = os.path.join(data_path,"vertices")

# Get the number of realizations
_, _, files = next(os.walk(trj_path))
realizations = len(files)

for i in range(1,5+1):
    print("========================================")
    print(f"Working on realization {i}")

    # Importing files
    trj_file = os.path.join(trj_path,f"trj{i}.csv")
    print(f"Opening " + trj_file)
    ctrj_file = os.path.join(ctrj_path,f"ctrj{i}.csv")
    print(f"Opening " + ctrj_file)
    trj_raw = trj = pd.read_csv(trj_file, index_col=[0,1])
    ctrj_raw = pd.read_csv(ctrj_file, index_col=[0,1])

    # Doing shit with the vertices
    v = ice.vertices()
    frames = ctrj_raw.index.get_level_values("frame").unique()

    vrt_file = os.path.join(vrt_path,f"vertices{i}.csv")
    v.trj_to_vertices(ctrj_raw.loc[frames[::2]])

    print(f"Saving vertices to " + vrt_file)
    v.vertices.to_csv(vrt_file)
    
    clear_output(wait=True)

In [ ]:
vrt_path = os.path.join(data_path,"vertices/")
vertices = aux.getVerticesDict(vrt_path)
counts = aux.getVerticesCount(vertices)
vrt_ts = aux.getVerticesAverage(counts,params["framespersec"])
types = vrt_ts.columns.to_list()[1:]
t = vrt_ts["time"].to_numpy()
vrt_counts = vrt_ts[types].to_numpy()

In [ ]:
field = t * params["max_field"].magnitude / quench_time.magnitude
params["max_field"]

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))

ax.plot(field,vrt_counts[:,0])
ax.plot(field,vrt_counts[:,1])
ax.plot(field,vrt_counts[:,2])
ax.plot(field,vrt_counts[:,3])
ax.plot(field,vrt_counts[:,4])
ax.plot(field,vrt_counts[:,5])

ax.legend(types, fontsize=12,title='Type', title_fontsize=15, edgecolor='black')
ax.set_xlabel("$B$  [ mT ] ", fontsize=15)
ax.set_ylabel("Vertices counts", fontsize=15)
ax.set_xlim(0,20)
ax.set_ylim(0,0.5)
plt.show()

fig.savefig('../data/vertices_full.pdf', bbox_inches='tight')
fig.savefig('../data/vertices_full.jpeg', dpi=300)

# Some visualizations

In [ ]:
region_limit = np.round( params["lattice_constant"].magnitude * params["size"] )
last_frame = trj_raw.index.get_level_values("frame").unique()[-1]
ctrj_sel = ctrj_raw.loc[last_frame,:]
v = ice.vertices()
v = v.trj_to_vertices(ctrj_sel)

In [ ]:
def drop_shit(ctrj):
    return ctrj.drop(columns={'type','t'})

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ice.draw_frame(drop_shit(ctrj_raw),
               region=[0,region_limit,0,region_limit],
               radius=params["particle_radius"].magnitude,
               cutoff=params["trap_sep"].magnitude/2,
               particle_color='#75b7ea',
               trap_color='gray',
               ax = ax)

v.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)
plt.show()

In [ ]:
ctrj_sel.to_csv('../data/test16/weirdshit.csv')

# Getting an AF coloidal thingie from previous state

Oh lord why am i doing this

In [ ]:
particle = ice.particle(radius = params["particle_radius"],
            susceptibility = params["particle_susceptibility"],
            diffusion = params["particle_diffusion"],
            temperature = params["particle_temperature"],
            density = params["particle_density"])

trap = ice.trap(trap_sep = params["trap_sep"],
            height = params["trap_height"],
            stiffness = params["trap_stiffness"])


params["particle"] = particle
params["trap"] = trap

col0 = aux.get_colloids_from_ctrj2(drop_shit(ctrj_sel),params)
col1 = col0.copy(deep=True)

flips = [190,192,194,195,197,198,181,185,94,95,
         170,172,173,174,175,178,179,83,84,
         73,74,164,
         150,153,155,156,157,158,
         141,144,145,146,147,149,54,55,56,57,
         130,131,133,134,135,139,40,43,44,45,49,
         122,123,32,33,34,
         112,114,115,22,23,24,
         100,102,103,104,105,106,107,108,109,12,13,14,15,
         0,1,2,3,4,5,6,7,8,9]
col1 = mc.flip_colloids(col1, indices = flips)

In [ ]:
v0 = ice.vertices()
v0 = v.colloids_to_vertices(col0)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10,24))

ax = axes[0]
col0.display(ax)
v0.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)

ax = axes[1]
v1 = ice.vertices()
v1 = v1.colloids_to_vertices(col1)
col1.display(ax)
v1.display(ax,dpl_scale=0.5,dpl_width=2.5,circle_scale=0.5)

In [ ]:
col1.to_ctrj().to_csv(os.path.join(data_path,'smolaf.csv'))

In [ ]:
field = [20,0,0]*ureg.mT 
E0 = col0.calculate_energy(B=field)
E1 = col1.calculate_energy(B=field)

print('Last frame energy:', E0)
print('AF energy:',E1)

# New simulations but, from the AF state

In [ ]:
col = col1
params['sim_temp'] = 300*ureg.K

world = ice.world(
        field = params["max_field"],
        temperature = params["sim_temp"],
        dipole_cutoff = params["sim_dipole_cutoff"],
        boundaries=['p', 'p', 'p'])
col.simulation(world,
            name = f"./lammps_files/test1",
            include_timestamp = False,
            targetdir = r".",
            framerate = params["framespersec"],
            timestep = params["dt"],
            run_time = params["total_time"],
            output = ["x","y","z","mux","muy","muz"],
            processors=1)

    # Field
    

col.sim.field.fieldx = "v_Bmag"
col.sim.field.fieldy = "0"
col.sim.field.fieldz = "0"        
        
col.run_simulation()

col.load_simulation()

In [ ]:
filename = f"trj.csv"
col.trj.to_csv(os.path.join(data_path,filename))

filename = f"ctrj.csv"
trj = ice.get_ice_trj(col.trj, bounds = col.bnd)
trj.to_csv(os.path.join(data_path,filename))